# Find Patients with DM2, Multiple Visits, and a Change in DM2 Diagnoses Over Time

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [46]:
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
admissions = pd.read_csv('ADMISSIONS.csv')

Any of the following ICD9 Codes would indicate that a patient has DM2

In [9]:
DMcodes = ['25000','25002','25010','25012','25020','25022','25030','25032',
         '25040','25042','25050','25052','25060','25062','25070','25072',
         '25080','25082''25090','25092','V5867', '25001','25003','25011','25013','25021','25023','25031','25033',
         '25041','25043','25051','25053','25061','25063','25071','25073',
         '25081','25083''25091','25093']

In [32]:
DMpatients = []

for index,row in diagnoses.iterrows():
    if(row['ICD9_CODE'] in DMcodes):
        DMpatients.append(row['HADM_ID'])
DMpatients = list(set(DMpatients))

In [35]:
masterDF = pd.DataFrame(columns=['hadm_id'])
for patient in DMpatients:
    new_row = pd.DataFrame([[patient]],columns=['hadm_id'])
    masterDF = masterDF.append(new_row)       

In [36]:
masterDF['subject_id'] = 0
masterDF = masterDF.reset_index(drop=True)

In [47]:
#print(admissions.head())
admissions = admissions.set_index('HADM_ID')
#print(admissions.head())

In [48]:
for index,row in masterDF.iterrows():
    patient = admissions.loc[row['hadm_id'],'SUBJECT_ID']
    masterDF.at[index, 'subject_id'] = patient

In [51]:
admissions = pd.read_csv('ADMISSIONS.csv')

In [52]:
timeDF = pd.DataFrame(columns=['hadm_id','time'])

for index,row in admissions.iterrows():
    if(row['HADM_ID'] in DMpatients):
        new_row = pd.DataFrame([[row['HADM_ID'],row['ADMITTIME']]],columns=['hadm_id','time'])
        timeDF = timeDF.append(new_row) 

In [53]:
masterDF = pd.merge(left=masterDF,right=timeDF, left_on='hadm_id', right_on='hadm_id')

In [54]:
masterDF['time']=pd.to_datetime(masterDF['time'])
masterDF = masterDF.sort_values(by=['time'])

In [55]:
#masterDF = pd.read_csv('masterPatient.csv')
#masterDF = masterDF.drop(['Unnamed: 0'],axis=1)
masterDF = masterDF.set_index('hadm_id')
masterDF['visit_num'] = 0

In [56]:
DMsubjects = []
for index,row in diagnoses.iterrows():
    if(row['ICD9_CODE'] in DMcodes):
        DMsubjects.append(row['SUBJECT_ID'])
DMsubjects = list(set(DMsubjects))

In [61]:
for patient in DMsubjects:
    df = masterDF.loc[masterDF['subject_id'] == patient]
    df['hadm_id'] = df.index
    df = df.reset_index(drop=True)
    for index,row in df.iterrows():                
        masterDF.at[row['hadm_id'],'visit_num'] = (index+1)

c:\users\ian\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
print(masterDF.head())

         subject_id                time  visit_num
hadm_id                                           
167070         4521 2100-06-28 19:29:00          1
137275         9319 2100-07-01 12:00:00          1
125380        31585 2100-07-02 19:28:00          1
132722         1426 2100-07-07 14:59:00          1
114310         6503 2100-07-11 06:15:00          1


In [63]:
masterDF.to_csv(r'newMasterPatient.csv')